<a href="https://colab.research.google.com/github/vahedshaik/cmpe255-assignment-6/blob/main/freqpatternmining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install apyori
!pip install mlxtend --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import pandas as pd
import numpy as np
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
     

READING DATASET

In [33]:
df = pd.read_csv("/content/Grocery Products Purchase.csv", header=None)

In [34]:

df.describe()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
count,9836,7677,6034,4735,3730,2875,2230,1685,1247,897,...,15,9,8,8,7,6,5,2,2,2
unique,159,152,156,154,151,138,139,141,129,121,...,14,8,7,8,6,5,4,2,2,2
top,sausage,whole milk,whole milk,whole milk,rolls/buns,soda,soda,shopping bags,soda,shopping bags,...,waffles,bottled beer,chocolate,Product 26,hygiene articles,napkins,house keeping products,Product 30,Product 31,Product 32
freq,825,654,506,315,176,150,120,76,61,49,...,2,2,2,1,2,2,2,1,1,1


CHECK FOR ANY NULL VALUES

In [35]:
df.isnull().sum()

0        0
1     2159
2     3802
3     5101
4     6106
5     6961
6     7606
7     8151
8     8589
9     8939
10    9185
11    9367
12    9484
13    9562
14    9639
15    9694
16    9740
17    9769
18    9783
19    9797
20    9806
21    9817
22    9821
23    9827
24    9828
25    9828
26    9829
27    9830
28    9831
29    9834
30    9834
31    9834
dtype: int64

REPLACE THE NULL VALUES WITH 0 FOR PREPOCESSING 

In [36]:
preprocessed = df.fillna(0)

In [37]:
preprocessed.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,Product 1,Product 2,Product 3,Product 4,Product 5,Product 6,Product 7,Product 8,Product 9,Product 10,...,Product 23,Product 24,Product 25,Product 26,Product 27,Product 28,Product 29,Product 30,Product 31,Product 32
1,citrus fruit,semi-finished bread,margarine,ready soups,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tropical fruit,yogurt,coffee,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,whole milk,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,pip fruit,yogurt,cream cheese,meat spreads,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
transactions = []
for i in range(0,len(preprocessed)):
    transactions.append([str(preprocessed.values[i,j]) for j in range(0,20) if str(preprocessed.values[i,j])!='0'])

In [39]:
len(transactions)

9836

In [40]:

transactions[:40]

[['Product 1',
  'Product 2',
  'Product 3',
  'Product 4',
  'Product 5',
  'Product 6',
  'Product 7',
  'Product 8',
  'Product 9',
  'Product 10',
  'Product 11',
  'Product 12',
  'Product 13',
  'Product 14',
  'Product 15',
  'Product 16',
  'Product 17',
  'Product 18',
  'Product 19',
  'Product 20'],
 ['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product'],
 ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'],
 ['rolls/buns'],
 ['other vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled beer',
  'liquor (appetizer)'],
 ['potted plants'],
 ['whole milk', 'cereals'],
 ['tropical fruit',
  'other vegetables',
  'white bread',
  'bottled water',
  'chocolate'],
 ['citrus fruit',
  'tropical fruit',
  'whole milk',
  'butter',
  'curd',
  'yogurt',
  'flo

**APRIORI**

In [41]:
rules = apriori(transactions,min_support=0.003,min_confidance=0.2,min_length=2)


In [42]:
Appriori_list = list(rules)


In [43]:
final_df = pd.DataFrame(Appriori_list)


In [44]:
final_df.head()


,items,support,ordered_statistics
0,(Instant food products),0.008032,"[((), (Instant food products), 0.0080317202114..."
1,(UHT-milk),0.033449,"[((), (UHT-milk), 0.03344855632370883, 1.0)]"
2,(abrasive cleaner),0.003457,"[((), (abrasive cleaner), 0.003456689711264741..."
3,(artif. sweetener),0.003253,"[((), (artif. sweetener), 0.003253355022366815..."
4,(baking powder),0.017588,"[((), (baking powder), 0.0175884505896706, 1.0)]"


**FBGROWTH SECTION**

In [45]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
t_df = pd.DataFrame(te_ary, columns=te.columns_)
t_df.head()
     

,Instant food products,Product 1,Product 10,Product 11,Product 12,Product 13,Product 14,Product 15,Product 16,Product 17,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [46]:
fpgrowth_df = fpgrowth(t_df, min_support=0.002, use_colnames=True)

In [47]:
fpgrowth_df.head()


,support,itemsets
0,0.082757,(citrus fruit)
1,0.058255,(margarine)
2,0.017690,(semi-finished bread)
3,0.139488,(yogurt)
4,0.104921,(tropical fruit)
